In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1892,2024-10-03,Dinamarca Liga De Basquete,14:00,Næstved,Randers,1.85,1.81,174.5,1.83,1.83,-1.5,1.97,1.71,x4tfUdMk,0.540541,0.552486,0.546448,0.546448,0.093027,305.130,169.041407,0.553998,0.6,1.341641,2.236068,89.89,3.558,1.914607,0.538113,-17.0,169.204,72.773930,0.430096,0.6,1.341641,2.236068,128.76,1.970,0.854810,0.433914,-22.0,89,74,1.01,1.74,170.122,287.332,0.015456,0.000000,0.099917,-2.50,-0.500,-1.700000,0.568550,0.5,-0.068550,6.26,1.252,0.646965,0.483192,0.6,0.116808
1893,2024-10-03,Países Baixos Wbl Feminina,15:00,Limburg F,QSTA United F,2.41,1.47,124.5,1.83,1.83,3.5,2.04,1.66,MBYh3C7T,0.414938,0.680272,0.546448,0.546448,0.095210,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,57,0,2.30,0.00,0.000,0.000,0.342619,0.000000,0.145244,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1894,2024-10-03,Japão B.League,07:05,Gunma,Hiroshima D.,1.49,2.38,157.5,1.88,1.78,-5.5,1.93,1.74,4pk7WM6D,0.671141,0.420168,0.531915,0.561798,0.091309,302.014,248.767278,0.823695,1.8,1.643168,0.912871,120.87,3.724,3.002038,0.806133,11.0,191.058,16.579363,0.086777,2.4,1.341641,0.559017,180.00,2.654,0.185822,0.070016,10.0,79,72,1.53,2.50,132.528,191.058,0.325233,0.038640,0.073215,-0.43,-0.086,-5.697674,0.668777,0.6,-0.068777,5.55,1.110,1.243243,0.446920,0.8,0.353080
1895,2024-10-03,Europa Supercopa Feminina,10:00,Fenerbahce F,Besiktas F,1.01,9.70,155.5,1.80,1.86,-22.5,1.94,1.77,IclLuba0,0.990099,0.103093,0.555556,0.537634,0.093192,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.147480,0.023184,0.064802,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1896,2024-10-03,Eslovênia Liga Otp Banka,14:00,Helios Domzale,Podcetrtek,1.19,3.93,159.5,1.84,1.82,-11.5,2.01,1.68,GYiRFTOF,0.840336,0.254453,0.543478,0.549451,0.094789,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,83,1.25,6.15,0.000,0.000,0.756825,0.007728,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1897,2024-10-03,Dinamarca Liga De Basquete,14:00,Værløse,Herlev Wolfpack,1.35,2.86,166.5,1.83,1.83,-6.5,1.95,1.72,UBb1Szi2,0.740741,0.349650,0.546448,0.546448,0.090391,383.560,268.854825,0.700946,0.6,1.341641,2.236068,149.24,4.632,3.118593,0.673271,-64.0,410.946,505.102476,1.229121,0.6,1.341641,2.236068,1312.00,5.086,6.129570,1.205185,-22.0,91,82,1.64,16.00,400.742,459.192,0.507236,0.000000,0.088629,-3.36,-0.672,-0.520833,0.375382,0.3,-0.075382,-5.00,-1.000,-1.860000,0.314691,0.0,-0.314691
1898,2024-10-03,Austrália Nbl,06:30,Adelaide 36ers,South East Melbourne,1.49,2.65,184.5,1.80,1.86,-5.5,2.00,1.72,nV7XiwOO,0.671141,0.377358,0.555556,0.537634,0.048499,223.412,65.626464,0.293746,1.2,1.643168,1.369306,321.48,2.606,0.605913,0.232507,-36.0,280.140,102.266532,0.365055,0.0,0.000000,NaN,176.96,3.724,1.731598,0.464983,-108.0,94,79,3.42,2.24,213.236,252.768,0.396253,0.023184,0.106446,3.67,0.734,0.667575,0.425621,0.7,0.274379,-5.00,-1.000,-1.650000,0.000000,0.0,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1900,10:00,Alemanha Bbl,Syntainics MBC,Heidelberg,1.83,Back Home
1905,14:00,Europa Euroliga,Monaco,Milano,1.56,Back Home
